In [2]:
import numpy as np
import tensorflow as tf
import keras


from groupy.gconv.tensorflow_gconv.splitgconv2d import gconv2d, gconv2d_util

2025-03-21 11:55:35.668390: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-21 11:55:35.798987: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742572535.848356     959 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742572535.862566     959 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1742572535.975186     959 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [3]:
(x_test, y_test), (x_train, y_train) = keras.datasets.mnist.load_data()

x_train = x_train.reshape(10000, 28, 28, 1).astype("float32") / 255
x_test = x_test.reshape(60000, 28, 28, 1).astype("float32") / 255

y_train = y_train.astype("float32")
y_test = y_test.astype("float32")

x_val = x_test[:2000]
y_val = y_test[:2000]
x_test = x_test[2000:]
y_test = y_test[2000:]


In [4]:
data_augmentation = keras.Sequential([
    keras.layers.RandomRotation(0.5)
])

x_train = data_augmentation(x_train)
x_test = data_augmentation(x_test)
x_val = data_augmentation(x_val)



I0000 00:00:1742572539.524654     959 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9712 MB memory:  -> device: 0, name: NVIDIA RTX A2000 12GB, pci bus id: 0000:01:00.0, compute capability: 8.6


In [5]:
y_train[:10]

array([7., 2., 1., 0., 4., 1., 4., 9., 5., 9.], dtype=float32)

# Z2 CNN Model

In [6]:
gconv2d_util("Z2", "C4", 1, 1, 3)

(array([[[[0, 1, 2],
          [3, 4, 5],
          [6, 7, 8]]],
 
 
        [[[2, 5, 8],
          [1, 4, 7],
          [0, 3, 6]]],
 
 
        [[[8, 7, 6],
          [5, 4, 3],
          [2, 1, 0]]],
 
 
        [[[6, 3, 0],
          [7, 4, 1],
          [8, 5, 2]]]], dtype=int32),
 (1, 4, 1, 1, 3),
 (3, 3, 1, 1))

In [7]:
# Define model
def make_z2cnn():
    z2cnn_model = keras.Sequential()

    z2cnn_model.add(keras.Input(shape=(28,28, 1)))

    for i in range(6):
        z2cnn_model.add(keras.layers.Conv2D(20,
                                            kernel_size=3,
                                            strides=1,
                                            activation="relu",
                                            name="conv" + str(i + 1)))
        z2cnn_model.add(keras.layers.Dropout(rate=0.3))
        if i == 1:
            z2cnn_model.add(keras.layers.MaxPooling2D())
    z2cnn_model.add(keras.layers.Conv2D(10,
                                        kernel_size=4,
                                        name="conv7"))

    z2cnn_model.add(keras.layers.GlobalMaxPool2D(keepdims=False))

    return z2cnn_model

make_z2cnn().summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1 (Conv2D)                  │ (None, 26, 26, 20)     │           200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 26, 26, 20)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2 (Conv2D)                  │ (None, 24, 24, 20)     │         3,620 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 24, 24, 20)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 12, 12, 20)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3 (Conv2D)                  │ (None, 10, 10, 20)     │         3,620 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 10, 10, 20)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv4 (Conv2D)                  │ (None, 8, 8, 20)       │         3,620 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 8, 8, 20)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv5 (Conv2D)                  │ (None, 6, 6, 20)       │         3,620 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 6, 6, 20)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv6 (Conv2D)                  │ (None, 4, 4, 20)       │         3,620 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 4, 4, 20)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv7 (Conv2D)                  │ (None, 1, 1, 10)       │         3,210 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling2d            │ (None, 10)             │             0 │
│ (GlobalMaxPooling2D)            │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,510 (84.02 KB)

 Trainable params: 21,510 (84.02 KB)

 Non-trainable params: 0 (0.00 B)

# P4 CNN Model

In [ ]:
class SplitConv2d(keras.Layer):
    h_input = None
    h_output = None
    input_stabilizer_size = None
    output_stabilizer_size = None
    def __init__(self,
                 out_channels:int,
                 kernel_size:int,
                 stride=1,
                 pad=0,
                 *,
                 activity_regularizer=None,
                 trainable=True,
                 dtype=np.float32,
                 autocast=True,
                 name=None,
                 **kwargs):
        super().__init__(activity_regularizer=activity_regularizer,
                         trainable=trainable,
                         dtype=dtype,
                         autocast=autocast,
                         name=name,
                         **kwargs)

        self.out_channels = out_channels
        self.kernel_size = kernel_size
        # self.stride = stride if hasattr(stride, "__getitem__") else (stride, stride)

    def build(self, input_shape):
        self.in_channels = input_shape[-1]
        
        inds, gconv_shape_info, w_shape = gconv2d_util(self.h_input,
                                                self.h_output,
                                                self.in_channels,
                                                self.out_channels,
                                                self.kernel_size)
        

        
        self.w = self.add_weight(
            shape=w_shape,
            name="W",
            dtype=self.dtype,
            initializer="random_normal"
        )

        self.b = self.add_weight(
            shape=(self.out_channels,),
            name='b',
            dtype=self.dtype
        )
        
 

    def call(self, inputs):
        inds, gconv_shape_info, w_shape = gconv2d_util(self.h_input,
                                        self.h_output,
                                        self.in_channels,
                                        self.out_channels,
                                        self.kernel_size)


        y = gconv2d(inputs,
                       self.w,
                       [1,1,1,1],
                       "VALID",
                       inds,
                       gconv_shape_info,
                       use_cudnn_on_gpu=True,
                       data_format='NHWC',)
        batch_size, _,  ny_out, nx_out = y.shape
        y = tf.reshape(y, (batch_size, self.out_channels, self.output_stabilizer_size, ny_out, nx_out))

        # bb = tf.reshape(self.b, (1, self.out_channels, 1, 1, 1))
        # b_shape = tf.broadcast_static_shape(y, bb)
        # y = tf.broadcast_to(y, b_shape)
        # b = tf.broadcast_to(bb, b_shape)

        # y = y + b

        # n, nc, ng, nx, ny = y.data.shape
        # y = tf.reshape(y, (n, nc * ng, nx, ny))
        # print(y.shape)
        return y



class P4ConvZ2(SplitConv2d):
    h_input = 'Z2'
    h_output = 'C4'
    input_stabilizer_size = 1
    output_stabilizer_size = 4


class P4ConvP4(SplitConv2d):
    h_input = 'C4'
    h_output = 'C4'
    input_stabilizer_size = 4
    output_stabilizer_size = 4
    

In [14]:
# Define model
def make_p4cnn():

    p4cnn_model = keras.Sequential()

    p4cnn_model.add(keras.Input(shape=(28,28, 1)))

    p4cnn_model.add(P4ConvZ2(10,
                            kernel_size=3,
                            stride=1,
                            name="conv1"))
    p4cnn_model.add(keras.layers.Activation('relu'))
    p4cnn_model.add(keras.layers.Dropout(rate=0.3))
    for i in range(2, 7):
        p4cnn_model.add(P4ConvP4(10,
                                kernel_size=3,
                                stride=1,
                                name="conv" + str(i)))
        
        p4cnn_model.add(keras.layers.Activation('relu'))
        p4cnn_model.add(keras.layers.Dropout(rate=0.3))
        if i == 2:
            p4cnn_model.add(keras.layers.MaxPooling2D())
    p4cnn_model.add(P4ConvP4(10,
                                        kernel_size=4,
                                        name="conv7"))

    return p4cnn_model

make_p4cnn().summary()


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1 (P4ConvZ2)                │ (None, 26, 26, 40)     │           100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_6 (Activation)       │ (None, 26, 26, 40)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ (None, 26, 26, 40)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2 (P4ConvP4)                │ (None, 24, 24, 40)     │         3,610 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_7 (Activation)       │ (None, 24, 24, 40)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_13 (Dropout)            │ (None, 24, 24, 40)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 12, 12, 40)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3 (P4ConvP4)                │ (None, 10, 10, 40)     │         3,610 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_8 (Activation)       │ (None, 10, 10, 40)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_14 (Dropout)            │ (None, 10, 10, 40)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv4 (P4ConvP4)                │ (None, 8, 8, 40)       │         3,610 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_9 (Activation)       │ (None, 8, 8, 40)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_15 (Dropout)            │ (None, 8, 8, 40)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv5 (P4ConvP4)                │ (None, 6, 6, 40)       │         3,610 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_10 (Activation)      │ (None, 6, 6, 40)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_16 (Dropout)            │ (None, 6, 6, 40)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv6 (P4ConvP4)                │ (None, 4, 4, 40)       │         3,610 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_11 (Activation)      │ (None, 4, 4, 40)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_17 (Dropout)            │ (None, 4, 4, 40)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv7 (P4ConvP4)                │ (None, 1, 1, 40)       │         6,410 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,560 (95.94 KB)

 Trainable params: 24,560 (95.94 KB)

 Non-trainable params: 0 (0.00 B)

In [19]:
def make_p4cnn_rotpool():

    model = keras.Sequential()

    model.add(keras.Input(shape=(28,28, 1)))

    model.add(P4ConvZ2(20,
                            kernel_size=3,
                            stride=1,
                            name="conv1"))
    model.add(keras.layers.Activation('relu'))
    model.add(keras.layers.maximum)
    # p4cnn_model.add(keras.layers.Dropout(rate=0.3))
    for i in range(2, 7):
        model.add(P4ConvZ2(20,
                                kernel_size=3,
                                stride=1,
                                name="conv" + str(i)))
        
        model.add(keras.layers.Activation('relu'))
        model.add(keras.layers.MaxPooling2D(pool_size=2, strides=2))
        # p4cnn_model.add(keras.layers.Dropout(rate=0.3))
        if i == 2:
            model.add(keras.layers.MaxPooling2D())
    model.add(keras.layers.P4ConvZ2(10,
                                        kernel_size=4,
                                        name="conv7"))

    model.add(keras.layers.Flatten())
    model.add(keras.layers.Softmax())

    return model

make_p4cnn_rotpool().summary()

ValueError: Only instances of `keras.Layer` can be added to a Sequential model. Received: <function maximum at 0x7ff37fd7af80> (of type <class 'function'>)

In [21]:
import time

In [23]:
epoch_num = 5
repeats = 1

models = [(make_z2cnn, epoch_num * 3), (make_p4cnn, epoch_num)]
results = {}
for model in models:
    results[model[0]] = []

starttime = time.time()

for i in range(repeats):
    tf.random.set_seed(i)
    for model_fn, epochs in models:
        model = model_fn()
        model.compile(
            optimizer=keras.optimizers.Adam(),
            loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
            metrics=[keras.metrics.sparse_categorical_accuracy]
        )
        model.fit(
            x_train,
            y_train,
            epochs=epochs,
            validation_data=(x_val, y_val)
        )
        results[model_fn].append(model.evaluate(x_test, y_test))

        del model

endtime = time.time()

Epoch 1/15
313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step - loss: 2.1660 - sparse_categorical_accuracy: 0.1917 - val_loss: 1.6096 - val_sparse_categorical_accuracy: 0.4810
Epoch 2/15
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 1.4787 - sparse_categorical_accuracy: 0.4710 - val_loss: 1.1716 - val_sparse_categorical_accuracy: 0.6445
Epoch 3/15
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 1.1071 - sparse_categorical_accuracy: 0.6232 - val_loss: 0.8635 - val_sparse_categorical_accuracy: 0.7535
Epoch 4/15
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.8876 - sparse_categorical_accuracy: 0.6925 - val_loss: 0.6982 - val_sparse_categorical_accuracy: 0.8010
Epoch 5/15
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.7760 - sparse_categorical_accuracy: 0.7460 - val_loss: 0.6083 - val_sparse_categorical_accuracy: 0.8255
Epoch 6/15
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.6754 - sparse_categorical_accuracy: 0.7764 - val_loss: 0.5319 - val_sparse_categorical_accuracy: 0.8410
Epo

/home/jquinteros/tfcode/tfenv/lib/python3.10/site-packages/keras/src/optimizers/base_optimizer.py:774: UserWarning: Gradients do not exist for variables ['sequential_22/conv1/b', 'sequential_22/conv2/b', 'sequential_22/conv3/b', 'sequential_22/conv4/b', 'sequential_22/conv5/b', 'sequential_22/conv6/b', 'sequential_22/conv7/b'] when minimizing the loss. If using `model.compile()`, did you forget to provide a `loss` argument?
  warnings.warn(


313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step - loss: 2.6444 - sparse_categorical_accuracy: 0.0915 - val_loss: 2.1117 - val_sparse_categorical_accuracy: 0.1660
Epoch 2/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 2.0025 - sparse_categorical_accuracy: 0.1797 - val_loss: 1.8128 - val_sparse_categorical_accuracy: 0.2120
Epoch 3/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 1.8023 - sparse_categorical_accuracy: 0.2060 - val_loss: 1.7390 - val_sparse_categorical_accuracy: 0.2320
Epoch 4/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 1.7010 - sparse_categorical_accuracy: 0.2367 - val_loss: 1.6460 - val_sparse_categorical_accuracy: 0.2385
Epoch 5/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 1.6568 - sparse_categorical_accuracy: 0.2285 - val_loss: 1.6104 - val_sparse_categorical_accuracy: 0.2370
1813/1813 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 1.6163 - sparse_categorical_accuracy: 0.2299


In [24]:
results

{<function __main__.make_z2cnn()>: [[0.31864604353904724, 0.9032413959503174]],
 <function __main__.make_p4cnn()>: [[1.615958333015442, 0.23108620941638947]]}

In [ ]:
results